<a href="https://colab.research.google.com/github/CallOfTheNight/CallOfTheNight/blob/main/NLE2023/Lab_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 3: Basic Document Classification (Part 1)

## Overview
In labs this week (and next), the focus will be on the application of sentiment analysis. You will be using a corpus of **movie reviews**.

You will be exploring various techniques that can be used to classify the sentiment of the movie reviews as either positive or negative.

You will be developing your own **Word List** and **Naïve Bayes** classifiers and then comparing them to the **NLTK Naïve Bayes** classifier.

First, we will need to download the movie_review corpus.

In [1]:
import nltk
nltk.download('movie_reviews')
nltk.download('stopwords')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

The movie_reviews corpus reader provides a number of useful methods:
   * .categories()
   * .fileids()
   * .words()
   
First, we can use `.categories()` to check the set of labels with which the reviews have been labelled

In [2]:
from nltk.corpus import movie_reviews

print(movie_reviews.categories())

['neg', 'pos']


We can use `.fileids()` to get all of the file names associated with a particular category.

In [3]:
pos_review_ids=movie_reviews.fileids('pos')
neg_review_ids=movie_reviews.fileids('neg')

print("The number of positive reviews is {}".format(len(pos_review_ids)))
print("The number of negative reviews is {}".format(len(neg_review_ids)))

The number of positive reviews is 1000
The number of negative reviews is 1000


We can use `.words()` to get back word-tokenised reviews.  The argument to `.words()` is the file id of an individual review.

In [4]:
print(movie_reviews.words(pos_review_ids[0]))

['films', 'adapted', 'from', 'comic', 'books', 'have', ...]


In [5]:
type(movie_reviews.words(pos_review_ids[0]))

nltk.corpus.reader.util.StreamBackedCorpusView

Note, the object returned by `movie_reviews.words()` looks a lot like a list (and behaves a lot like a list) - but it is actually a `StreamBackedCorpusView`.  This essentially means it is not necessarily all in memory  - it is retrieved from disk as needed.  If you want to see all of the words at once then you can convert it to a list using the `list()` constructor.  

In [6]:
print(list(movie_reviews.words(pos_review_ids[0])))

['films', 'adapted', 'from', 'comic', 'books', 'have', 'had', 'plenty', 'of', 'success', ',', 'whether', 'they', "'", 're', 'about', 'superheroes', '(', 'batman', ',', 'superman', ',', 'spawn', ')', ',', 'or', 'geared', 'toward', 'kids', '(', 'casper', ')', 'or', 'the', 'arthouse', 'crowd', '(', 'ghost', 'world', ')', ',', 'but', 'there', "'", 's', 'never', 'really', 'been', 'a', 'comic', 'book', 'like', 'from', 'hell', 'before', '.', 'for', 'starters', ',', 'it', 'was', 'created', 'by', 'alan', 'moore', '(', 'and', 'eddie', 'campbell', ')', ',', 'who', 'brought', 'the', 'medium', 'to', 'a', 'whole', 'new', 'level', 'in', 'the', 'mid', "'", '80s', 'with', 'a', '12', '-', 'part', 'series', 'called', 'the', 'watchmen', '.', 'to', 'say', 'moore', 'and', 'campbell', 'thoroughly', 'researched', 'the', 'subject', 'of', 'jack', 'the', 'ripper', 'would', 'be', 'like', 'saying', 'michael', 'jackson', 'is', 'starting', 'to', 'look', 'a', 'little', 'odd', '.', 'the', 'book', '(', 'or', '"', 'grap

## Creating training and testing sets
You will be training and testing various document classifiers. It is essential that the data used in the testing phase is not used during the training phase, since this can lead to overestimating performance.

We now introduce the `split_data` function (defined in the cell below) which can be used to get separate **training** and **testing** sets.

> Look through the code in the following cell, reading the comments and making sure that you understand each line.

In [7]:
import random # have a look at the documentation at https://docs.python.org/3/library/random.html


def split_data(data, ratio=0.7): # when the second argument is not given, it defaults to 0.7
    """
    Given collection of items and ratio:
     - partitions the collection into training and testing, where the proportion in training is ratio,

    :param data: A list (or generator) of documents or doc ids
    :param ratio: The proportion of training documents (default 0.7)
    :return: a pair (tuple) of lists where the first element of the
            pair is a list of the training data and the second is a list of the test data.
    """

    n = len(data)  #Found out number of samples present.  data could be a list or a generator
    train_indices = random.sample(range(n), int(n * ratio))          #Randomly select training indices
    test_indices = list(set(range(n)) - set(train_indices))   #Other items are testing indices

    train = [data[i] for i in train_indices]           #Use training indices to select data
    test = [data[i] for i in test_indices]             #Use testing indices to select data

    return (train, test)                       #Return split data


Now we can use this function to create training and testing data.  First, we need to create 4 lists:
    * file ids  of positive docs to go in the training data
    * file ids of positive docs to go in the testing data
    * file ids of negative docs to go in the training data
    * file ids of negative docs to go in the testing data

In [8]:
random.seed(41)  #set the random seeds so these random splits are always the same
pos_train_ids, pos_test_ids = split_data(pos_review_ids)
neg_train_ids, neg_test_ids = split_data(neg_review_ids)


Now, we want to create our labelled data sets.   We need to associate each review with its label so that later we can shuffle up all of the training data (and the testing data)

### Exercise 1
Write some python code which will construct a training set (`training`) and a test set (`testing`) from the data.  Each set should be a list of pairs where each pair is a list of words and a label, as below:

<code>[([list,of,words],'label'),([list,of,words],'label'),...]</code>

Hint:  You can do this with 4 list comprehensions and list concatenation.

Check the size of `training` and `testing`.  Using a 70\% split, how many should be in each?

In [95]:
# list(movie_reviews.words(pos_review_ids[0]))

training = []
testing = []

for i in pos_train_ids:
  training.append((list(movie_reviews.words(i)),"pos"))

for i in neg_train_ids:
  training.append((list(movie_reviews.words(i)),"neg"))

print(len(training))

for i in pos_test_ids:
  testing.append((list(movie_reviews.words(i)),"pos"))

for i in neg_test_ids:
  testing.append((list(movie_reviews.words(i)),"neg"))

# print(testing[0])
training = list(training)
testing = list(testing)
# split

1400


## Document Representations

Currently, each review / document is represented as a list of tokens.  In many simple applications, the order of words in a document is deemed irrelevant and we use a bag-of-words representation of the document.  We can create a bag-of-words using a dictionary (as we did in Lab_2_2 when considering the size of the vocabulary) or we can use a library function such as FreqDist from nltk.probability (or Counter from Collections).  In the cell below, I generate the bag-of-words for the first review in the training set using nltk's FreqDist.  You can think of this as like a dictionary but with extra benefits.  For example, later on in the lab, we will see it has useful methods which allow the document representations to be added and subtracted.

In [41]:
from nltk.probability import FreqDist

doc1 = FreqDist(training[0][0])
doc1

FreqDist({',': 24, '.': 18, 'and': 11, 'a': 9, 'to': 8, 'the': 7, 'melvin': 6, 'his': 6, "'": 6, 's': 6, ...})

### Exercise 2.1

Write code to use FreqDist to construct a bag-of-words representation for each document in the training and testing sets.  Store the results in two lists, `training_basic` and `testing_basic`.  Don't lose the annotations as to whether each review is positive or negative!  

In [56]:
training_basic = []
testing_basic = []

for i in range(len(training)):
  training_basic.append((FreqDist(training[i][0]),training[i][1]))

for i in range(len(testing)):
  testing_basic.append((FreqDist(testing[i][0]),testing[i][1]))

print(list(testing_basic[2]))

[FreqDist({'the': 33, ',': 28, '.': 17, "'": 11, 'it': 10, 'of': 10, 'a': 9, 'and': 9, 'as': 9, 'to': 7, ...}), 'pos']


You will notice of course that many of the words in your representations of documents are punctuation and stopwords.  This is because we haven't done any pre-processing of the wordlists.

### Exercise 2.2

Decide which of the following pre-processing steps to apply to the word lists:-
* case normalisation
* number normalisation
* punctuation removal
* stopword removal
* stemmming / lemmatisation


Apply these preprocessing steps to the original wordlist representations (stored in `training` and `testing`).  Then recreate the bag-of-words representations, storing the results in `training_norm` and `testing_norm`

In [128]:
punctuation = [",",".","!","?","_",":","'","-","(",")","/"," ",""]

for i in range(len(training)):
  for j in range(len(training[i][0])):

    # element: training[i][0][j]

    # case normalisation
    training[i][0][j] = training[i][0][j].lower()

    # number normalisation
    if(training[i][0][j].isdigit()):
      training[i][0][j] = "NUM"

    # punctuation removal
    if training[i][0][j] in punctuation:
      del training[i][0][j]

  print(training[i][0])






['melvin', 'udall', 'is', 'a', 'heartless', 'man', 'he', 'spends', 'his', 'days', 'inside', 'of', 'his', 'spacious', 'manhattan', 'apartment', 'writing', 'romance', 'novels', 'it', 'also', 'seems', 'that', 'melvin', 'will', 'never', 'change', 'one', 'day', 'he', 'dines', 'ar', 'his', 'favorite', 'restaurant', 'and', 'is', 'a', 'little', 'too', 'mean', 'to', 'his', 'normal', 'waitress', 'the', 'only', 'waittress', 'that', 'will', 'serve', 'him', 'carol', 'played', 'to', 'perfection', 'by', 'a', 'lovely', 'and', 'sexy', 'helen', 'hunt', 'she', 'threatens', 'not', 'to', 'serve', 'him', 'if', 'he', 'doesn', 't', 'shut', 'up', 'about', 'her', 'asthmatic', 'son', 'he', 'does', 'shut', 'up', 'to', 'make', 'matters', 'considerably', 'worse', 'melvin', 'has', 'obsessive', 'compulsive', 'disorder', 'one', 'day', 'his', 'gay', 'artist', 'neighbor', 'simon', 'greg', 'kinear', 'of', 'talk', 'soup', 'fame', 'in', 'an', 'oscar', 'worthy', 'role', 's', 'dog', 'threatens', 'to', 'dismiss', 'himself', '

IndexError: ignored

## Creating word lists
The next section will explain how to use a sentiment classifier that bases its decisions on word lists. The classifier requires a list of words indicating positive sentiment, and a second list of words indicating negative sentiment. Given positive and negative word lists, a document's overall sentiment is determined based on counts of occurrences of words that occur in the two lists. In this section we are concerned with the creation of the word lists. We will be considering both hand-crafted lists and automatically generated lists.

### Exercise 3.1

- Create a reasonably long hand-crafted list of words that you think indicate positive sentiment.
- Create a reasonably long hand-crafted list of words that indicate negative sentiment.

Use the following cells to store these lists in the variables `my_positive_word_list` and `my_negative_word_list`.

In [ ]:
my_positive_word_list = ["good","great","lovely"] # extend this one or put your own list here
my_negative_word_list = ["bad", "terrible", "awful"] # extend this one or put your own list here

Now lets see how often each of those words occurs in total in our positive and negative training data.  First, lets create a total of the FreqDists for positive data and for negative data.  As these are FreqDists (rather than simple dictionaries), we can do this as follows:

In [ ]:
pos_freq_dist=FreqDist()
neg_freq_dist=FreqDist()

for reviewDist,label in training_norm:
    if label=='pos':
        pos_freq_dist+=reviewDist
    else:
        neg_freq_dist+=reviewDist

pos_freq_dist

### Exercise 3.2
In the blank code cell below write code that uses the total frequency distributions `pos_freq_dist` and `neg_freq_dist` and the word lists `my_positive_word_list` and `my_negative_word_list` created earlier to determine whether or not the review data conforms to your expectations. In particular, whether:
- the words you expected to indicate positive sentiment actually occur more frequently in positive reviews than negative reviews
- the words you expected to indicate negative sentiment actually occur more frequently in negative reviews than positive reviews.

You could display your findings in a table using pandas.

### Exercise 3.3
Now, you are going to create positive and negative word lists automatically from the training data. In order to do this:

1. write two new functions to help with automating the process of generating wordlists.

    - `most_frequent_words` - this function should take THREE arguments: 2 frequency distributions and a natural number, k. It should order words by how much more they occur in one frequency distribution than the other.   It should then return the top k highest scoring words. You might want to use the `most_common` method from the `FreqDist` class - this returns a list of word, frequency pairs ordered by frequency.  You might also or alternatively want to use pythons built-in `sorted` function
    - `words_above_threshold` - this function also takes three arguments: 2 frequency distributions and a natural number, k. Again, it should order words by how much more they occur in one distribution than the other.  It should return all of the words that have a score greater than k.

2. Using the training data, create two sets of positive and negative word lists using these functions (1 set with each function).
3.  Display these 4 lists (possibly in a `Pandas` dataframe?)



## Creating a word list based classifier
Now you have a number of word lists for use with a classifier.
> Make sure you understand the following code, which will be used as the basis for creating a word list based classifier.

In [ ]:
from nltk.classify.api import ClassifierI
import random

class SimpleClassifier(ClassifierI):

    def __init__(self, pos, neg):
        self._pos = pos
        self._neg = neg

    def classify(self, words):
        score = 0

        # add code here that assigns an appropriate value to score
        return "neg" if score < 0 else "pos"

    ##we don't actually need to define the classify_many method as it is provided in ClassifierI
    #def classify_many(self, docs):
    #    return [self.classify(doc) for doc in docs]

    def labels(self):
        return ("pos", "neg")

#Example usage:

classifier = SimpleClassifier(my_positive_word_list, my_negative_word_list)
classifier.classify(FreqDist("This movie was great".split()))

### Exercise 3.1

- Copy the above code cell and move it to below this one. Then complete the `classify` method in the above code as specified below.
- Test your classifier on several very simple hand-crafted examples to verify that you have implemented `classify` correctly.

The classifier is initialised with a list of positive words, and a list of negative words. The words of a document are passed to the `classify` method (which is partially completed in the above code fragment). The `classify` method should be defined so that each occurrence of a negative word decrements `score`, and each occurrence of a positive word increments `score`.
- For `score` less than 0, an "`N`" for negative should be returned.
- For `score` greater than 0,  "`P`" for positive should returned.
- For `score` of 0, the classification decision should be made randomly (see https://docs.python.org/3/library/random.html).


### Exercise 3.2
* Extend your SimpleClassifier class so that it has a `train` function which will derive the wordlists from training data.  You could build a separate class for each way of automatically deriving wordlists (which both inherit from SimpleClassifier) OR a single class which takes an extra parameter at training time.

Try out your classifier on the test data.  We will look at how to evaluate classifiers in the next part, but in an ideal world, most of the positive test items will have been classified as 'P' and most of the negative test items will have been classified as 'N'.  Note that the batch_classify method takes a list of unlabelled documents so you can't give it a list of pairs (where each pair is doc and a label).  You can either use a list comprehension or the <code>zip(*list_of_pairs)</code> function to split a list of pairs into a pair of lists.